In [2]:
import sqlite3
# DBMS(일반적으로 서버) - DB 연결 중, 우리는 client라서 접근(connection)해야함.
# cursor ---> 작업공간
# con - cur

# 첫번째로 connetion이 필요하다.
con = sqlite3.connect(':memory:') # file path or in memory에서 가능, 이번 경우엔 in memory에서 사용

In [36]:
# cursor를 부여 받아야함.
cur = con.cursor()

In [6]:
dir(cur)
# 'execute'와 관련된 것이 SQL을 커서를 통해서 실행한다.

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'arraysize',
 'close',
 'connection',
 'description',
 'execute',
 'executemany',
 'executescript',
 'fetchall',
 'fetchmany',
 'fetchone',
 'lastrowid',
 'row_factory',
 'rowcount',
 'setinputsizes',
 'setoutputsize']

In [ ]:
# execute를 통해 SQL구문이 1개 들어간다.
cur.execute('SQL문 한 개')
cur.executemany('SQL문 한 개가 반복실행')
# 비표준 방법이지만 편의성을 위해 사용한다.
# 중간에 오류가 나면 멈춘다.
cur.executescript('SQL문 여러개 한번에 실행')

In [37]:
cur.execute('''
    CREATE TABLE CITY (
    CNO INTEGER PRIMARY KEY,
    CNAME TEXT
    );
''')

In [38]:
cur.execute('''
    INSERT INTO CITY (CNO, CNAME)
    VALUES(1, "London");
''')

In [10]:
# dtype이 잘못들어가면 오류가 발생한다.
cur.execute('''
    INSERT INTO CITY (CNO, CNAME)
    VALUES('dad', "London");
''')

IntegrityError: datatype mismatch

In [39]:
cur.execute('''
    INSERT INTO CITY
    VALUES(2, 'Paris')
''')

In [40]:
# CNO는 primary key라서 3이 입력된다.
cur.execute('''
    INSERT INTO CITY (CNAME)
    VALUES('Rome')
''')

In [41]:
# CNO는 primary key라 not null인데 null을 넣으면 어떻게 될까?
# primary key는 Not null과 Unique가 기본 조건이기 때문에 다음 순서의 값이 배정된다.
# 따라서 values는 (4, 'Vienna')가 삽입될 것이다.
cur.execute('''
    INSERT INTO CITY 
    VALUES(NULL, "Vienna")
''')

In [ ]:
# fetch는 cursor를 이용하여 데이터를 가져온다. 
# 사용하기 위해서는 cursor는 데이터를 가지고 있어야한다.
cur.fetchone('Tuple/Record/Row 1개 가져온다')
cur.fetchmany('N개')
cur.fetchall('전부')

In [42]:
cur.execute('SELECT * FROM CITY')

In [43]:
# iterate라서 한개씩 추출된다.
# cur.fetchone()
# cur.fetchmany(2)
cur.fetchall()

[(1, 'London'), (2, 'Paris'), (3, 'Rome'), (4, 'Vienna')]

In [45]:
cur.execute('''
    CREATE TABLE TEMP(
    PK INTEGER NOT NULL UNIQUE,
    NAME CHAR(5) NOT NULL
    )
''')

In [46]:
# NOT NULL을 넣어서 INSERT에 NULL이 안된다.
cur.execute('INSERT INTO TEMP VALUES(NULL, "ABCD")')

IntegrityError: NOT NULL constraint failed: TEMP.PK

In [47]:
# sqlite에서는 문자열은 text로 받아들이기에 기존조건인 char(5)가 무시된다.
cur.execute('INSERT INTO TEMP VALUES(1, "ABCDWEE")')

In [48]:
cur.execute('SELECT * FROM TEMP')

In [50]:
cur.fetchone()

In [63]:
cur.execute('''drop table SUPPLIER''')

In [68]:
# supplier의 조건은 sno을 primary key, sname은 not null에 값이 지정되지 않으면 'NONAME'
cur.execute('''
    CREATE TABLE SUPPLIER(
    SNO INTEGER PRIMARY KEY,
    SNAME TEXT NOT NULL DEFAULT 'NONAME',
    CNO INTEGER NOT NULL
    )
''')

In [69]:
# 특정 값을 넣는 방법
# qmark = ?, named = k:v
# 값이 지정되지 않았을 때 ?를 통해 넣는다.
sname = 'Smith'
cno = 1

# 아래 구문은 qmark 스타일에 대입 방법이다. ?는 값을 넣는 것이고, 콤마는 위치를 의미한다.
cur.execute('''
    INSERT INTO SUPPLIER VALUES(NULL, ?, ?);
''', (sname, cno))

In [70]:
cur.execute('SELECT * FROM SUPPLIER')
cur.fetchall()

[(1, 'Smith', 1)]

In [71]:
# 데이터를 넣기 위해서 ?, ?에 값을 대입한다.
# 값을 삽입하기 위해 TUPLE에 데이터 PAIR를 만든다.
data = (('Jones',2), ('Adams', 1), ('Blake',3))
# 여러개의 데이터를 넣기 위해 executemany를 사용함.
cur.executemany('''
   INSERT INTO SUPPLIER VALUES(NULL, ?, ?);
''', data)

In [72]:
cur.execute('SELECT * FROM SUPPLIER')
cur.fetchall()

[(1, 'Smith', 1), (2, 'Jones', 2), (3, 'Adams', 1), (4, 'Blake', 3)]

In [80]:
cur.execute('''drop table part''')

In [59]:
cur.execute('''
    CREATE TABLE PART(
    PNO INTEGER PRIMARY KEY,
    PNAME TEXT
    )
''')

In [60]:
# named 방식의 데이터 기입 방식이다.
cur.execute('''
    INSERT INTO PART
    VALUES (:pno, :pname)
''', {'pno':1, 'pname':'Screw'})

In [61]:
cur.execute('SELECT * FROM PART')
cur.fetchall()

[(1, 'Screw')]

In [62]:
data = [ {'pname':'Nut'},
         {'pname':'Bolt'},
         {'pname':'Cam'}]

cur.executemany('''
    INSERT INTO PART
    VALUES (Null, :pname)
''', data)

In [63]:
cur.execute('SELECT * FROM PART')
cur.fetchall()

[(1, 'Screw'), (2, 'Nut'), (3, 'Bolt'), (4, 'Cam')]

In [64]:
cur.execute('''
    CREATE TABLE SELLS(
    SNO INTEGER NOT NULL,
    PNO INTEGER NOT NULL,
    PRICE INTEGER NOT NULL
    )
''')

In [73]:
key1 = 'Screw'
key2 = 'Smith'
pno = 0 
sno = 0 

cur.execute('SELECT PNO FROM PART WHERE PNAME = ?', [key1])

result = cur.fetchone()
if result:
    # 결과가 있을 때
    pno = result[0]

cur.execute('SELECT SNO FROM SUPPLIER WHERE SNAME = ?', [key2])

result = cur.fetchone()
if result:
    # 결과가 있을 때
    sno = result[0]

pno, sno

(1, 1)

In [74]:
data = ['Smith', 'Screw', 10]
# 중첩 SQL 나중에 procesure로 만든다.
# 여러개가 있을 때 LIMIT문을 이용하여 하나를 가져온다. 
cur.execute('''
    INSERT INTO SELLS
    VALUES(
          (SELECT SNO FROM SUPPLIER WHERE SNAME = ? LIMIT 0, 1), 
          (SELECT PNO FROM PART WHERE PNAME = ? LIMIT 0, 1),
    ?
    )
''', data)

In [75]:
cur.execute('SELECT * FROM SELLS')
cur.fetchall()

[(1, 1, 10)]

In [97]:
CITY = dict()

cur.execute('SELECT * FROM CITY')
for row in cur.fetchall():
    CITY[row[1]] = row[0]

In [98]:
CITY['London']

1

In [102]:
data = [(1,2,8),
        (2,4,38),
        (3,1,11),
        (3,3,6),
        (4,2,7),
        (4,3,4),
        (4,4,45)]
cur.executemany('INSERT INTO SELLS VALUES(?,?,?)', data)

In [105]:
# DB 73 page 내용
cur.execute('SELECT * FROM SELLS WHERE PRICE > 11')
cur.fetchall()

[(2, 4, 38), (4, 4, 45)]

In [109]:
# CROSS JOIN
cur.execute('SELECT * FROM SUPPLIER, PART;')
cur.fetchall()

[(1, 'Smith', 1, 1, 'Screw'),
 (1, 'Smith', 1, 2, 'Nut'),
 (1, 'Smith', 1, 3, 'Bolt'),
 (1, 'Smith', 1, 4, 'Cam'),
 (2, 'Jones', 2, 1, 'Screw'),
 (2, 'Jones', 2, 2, 'Nut'),
 (2, 'Jones', 2, 3, 'Bolt'),
 (2, 'Jones', 2, 4, 'Cam'),
 (3, 'Adams', 1, 1, 'Screw'),
 (3, 'Adams', 1, 2, 'Nut'),
 (3, 'Adams', 1, 3, 'Bolt'),
 (3, 'Adams', 1, 4, 'Cam'),
 (4, 'Blake', 3, 1, 'Screw'),
 (4, 'Blake', 3, 2, 'Nut'),
 (4, 'Blake', 3, 3, 'Bolt'),
 (4, 'Blake', 3, 4, 'Cam')]

In [112]:
# INNER JOIN
# 오름차순은 ASC, 내림차순은 DESC
cur.execute('SELECT B.CNAME, A.SNAME FROM SUPPLIER A, CITY B WHERE A.CNO = B.CNO ORDER BY A.CNO ASC;')
cur.fetchall()

[('London', 'Smith'),
 ('London', 'Adams'),
 ('Paris', 'Jones'),
 ('Rome', 'Blake')]

In [115]:
# '''은 줄바꿈을 사용하기 위해서 사용함. 한줄로 사용할 경우 ' 하나로 가능함.
cur.execute('''
            SELECT B.CNAME, COUNT(*) 
            FROM SUPPLIER A, CITY B 
            WHERE A.CNO = B.CNO 
            GROUP BY A.CNO 
            ORDER BY A.CNO ASC;
            ''')
cur.fetchall()

[('London', 2), ('Paris', 1), ('Rome', 1)]

In [121]:
cur.execute('''
            SELECT SNAME, PNAME, PRICE --COUNT(*)
            FROM SELLS, SUPPLIER, PART -- CROSS JOIN하면 8*4*4개의 값이 나온다.
            WHERE SELLS.SNO = SUPPLIER.SNO
            AND SELLS.PNO = PART.PNO
            ORDER BY PRICE DESC
            LIMIT 1,3 ; -- LIMIT는 순서 중 2번째부터 4번째 ROW를 들고옴(0부터 시작)
            ''')
cur.fetchall()

[('Jones', 'Cam', 38), ('Adams', 'Screw', 11), ('Smith', 'Screw', 10)]

In [124]:
cur.execute('''
            SELECT PNAME, COUNT(*), SUM(PRICE) -- 부품별 거래 이력 및 가격합
            FROM SELLS, SUPPLIER, PART -- CROSS JOIN하면 8*4*4개의 값이 나온다.
            WHERE SELLS.SNO = SUPPLIER.SNO
            AND SELLS.PNO = PART.PNO
            GROUP BY PART.PNO
            ORDER BY PRICE DESC
            --LIMIT 1,3 ; -- LIMIT는 순서 중 2번째부터 4번째 ROW를 들고옴(0부터 시작)
            ''')
cur.fetchall()

[('Cam', 2, 83), ('Screw', 2, 21), ('Nut', 2, 15), ('Bolt', 2, 10)]

In [128]:
cur.execute('''
            SELECT SNAME, COUNT(*), SUM(PRICE) -- 부품별 거래 이력 및 가격합
            FROM SELLS, SUPPLIER, PART -- CROSS JOIN하면 8*4*4개의 값이 나온다.
            WHERE SELLS.SNO = SUPPLIER.SNO
            AND SELLS.PNO = PART.PNO
            GROUP BY SUPPLIER.SNO
            ORDER BY SUM(PRICE) DESC
            --LIMIT 1,3 ; -- LIMIT는 순서 중 2번째부터 4번째 ROW를 들고옴(0부터 시작)
            ''')
cur.fetchall()

[('Blake', 3, 56), ('Jones', 1, 38), ('Smith', 2, 18), ('Adams', 2, 17)]

In [172]:
# 작업이 완료되었을 때 닫는 방법
con.close()

In [3]:
con = sqlite3.connect(r'C:\Users\GyeongJun\Desktop\고대_SW교육\오전\0905.db')
cur = con.cursor()

In [4]:
cur.execute('drop table part')

In [5]:
cur.executescript('''
    CREATE TABLE PART(
    PNO INTEGER PRIMARY KEY,
    PNAME TEXT
    )
''')

In [6]:
# create 이후 처음의 값이 기존 DB의 값이 된다.
cur.executescript('''
    INSERT INTO PART VALUES(1, 'Screw');
    INSERT INTO PART(PNAME) VALUES('Screw');
    INSERT INTO PART VALUES(NULL, 'Nut');
    INSERT INTO PART VALUES(4, 'Cam');
''')

In [7]:
# 이후의 값은 반영할지 안할지 정하지 않음.
# DB Lock이 걸리면 vscode(SQLite를 조회하던 프로그램을 껐다가 다시 킨다.)
cur.execute('''INSERT INTO PART VALUES(NULL, 'Part1');''')

In [8]:
cur.execute('SELECT * FROM PART')
cur.fetchall()

[(1, 'Screw'), (2, 'Screw'), (3, 'Nut'), (4, 'Cam'), (5, 'Part1')]

In [10]:
# commit을 해주면 적용이 안되는 insert문이 반영된다.
con.commit()

In [11]:
cur.execute('SELECT * FROM PART')
cur.fetchall()

[(1, 'Screw'), (2, 'Screw'), (3, 'Nut'), (4, 'Cam'), (5, 'Part1')]

In [12]:
con.close()

In [13]:
# 따라서 값을 저장하는 것을 저장하지 않고 close하지 않고 다시 실행하니 값이 저장되어 있지 않음.
con = sqlite3.connect(r'C:\Users\GyeongJun\Desktop\고대_SW교육\오전\0905.db')
cur = con.cursor()
cur.execute('SELECT * FROM PART')
cur.fetchall()

[(1, 'Screw'), (2, 'Screw'), (3, 'Nut'), (4, 'Cam'), (5, 'Part1')]

In [16]:
# UPDATE 구문
cur.execute('''
   UPDATE PART
   SET PNAME = '아무거나'         
''')
cur.execute('SELECT * FROM PART')
cur.fetchall()

[(1, '아무거나'), (2, '아무거나'), (3, '아무거나'), (4, '아무거나'), (5, '아무거나')]

In [18]:
cur.execute('INSERT INTO PART VALUES(NULL, "ㅇㅇㅇㅇ")')

In [20]:
cur.lastrowid # 맹신 금지

6

In [21]:
cur.execute('SELECT * FROM PART')
cur.fetchall()[-1]

(6, 'ㅇㅇㅇㅇ')

In [22]:
con.close()

In [23]:
con = sqlite3.connect(r'C:\Users\GyeongJun\Desktop\고대_SW교육\오전\0905.db')
cur = con.cursor()

In [25]:
# UNIQUE 조건의 오류
cur.executescript('''
    DROP TABLE IF EXISTS TEMP; -- IF TEMP문이 존재하면 지우고 테이블 새로 만들기
    CREATE TABLE TEMP(
    PK INTEGER NOT NULL UNIQUE,
    NAME TEXT NOT NULL
    );
     
    INSERT INTO TEMP VALUES(1, '테스트1');
    INSERT INTO TEMP VALUES(2, '테스트2');
    INSERT INTO TEMP VALUES(3, '테스트3');         
''')

In [26]:
# 제약 조건들을 백업하는 것
# dump란 DB상태를 저장하고 불러오는 툴이다.
for _ in con.iterdump():
    print(_)
# Begin Transaction에서 문제가 생기면 초기화된다.

BEGIN TRANSACTION;
CREATE TABLE PART(
    PNO INTEGER PRIMARY KEY,
    PNAME TEXT
    );
INSERT INTO "PART" VALUES(1,'Screw');
INSERT INTO "PART" VALUES(2,'Screw');
INSERT INTO "PART" VALUES(3,'Nut');
INSERT INTO "PART" VALUES(4,'Cam');
INSERT INTO "PART" VALUES(5,'Part1');
CREATE TABLE TEMP(
    PK INTEGER NOT NULL UNIQUE,
    NAME TEXT NOT NULL
    );
INSERT INTO "TEMP" VALUES(1,'테스트1');
INSERT INTO "TEMP" VALUES(2,'테스트2');
INSERT INTO "TEMP" VALUES(3,'테스트3');
COMMIT;


In [27]:
# ORM 테이블 만들기
con = sqlite3.connect(r'C:\Users\GyeongJun\Desktop\고대_SW교육\오전\sns.db')
cur = con.cursor()

In [95]:
cur.executescript('''
    DROP TABLE IF EXISTS POST;
    CREATE TABLE POST(
        PK INTEGER PRIMARY KEY,
        CONTENT TEXT
    );
    DROP TABLE IF EXISTS HASHTAG;
    CREATE TABLE HASHTAG(
        PK INTEGER PRIMARY KEY,
        COUNT INTEGER DEFAULT 0,
        NAME TEXT
    );
    DROP TABLE IF EXISTS POSTTAG;
    CREATE TABLE POSTTAG(
        PPK INTEGER NOT NULL,
        HPK INTEGER NOT NULL
    );
''')

In [92]:
# HASHTAG POOL <= TAG가 있는지 검사하는 모듈 만들기
# 있으면, 해당 PK값을 가져오고, 없으면 새롭게 INSERT함
data = {'tag':'Python'}

# 이부분이 tag 존재 유무 확인
cur.execute('''
            SELECT COUNT(*) 
            FROM HASHTAG 
            WHERE NAME = :tag -- :뒤에 값이 key값이다.
            ''', data)
# 만약 tag가 없다면 Hashtag table name에 tag값을 넣는다.
if cur.fetchone()[0] == 0:
    cur.execute('''
         INSERT INTO HASHTAG(NAME) VALUES(:tag)
                ''', data)
    # DB 커밋
    con.commit()
    
cur.execute('''
            SELECT PK 
            FROM HASHTAG 
            WHERE NAME = :tag 
            LIMIT 0,1 -- 여러개가 있을 수 있으니 한 개만 추출
            ''', data)
PK = cur.fetchone()[0]
PK

1

In [97]:
# Posting 하는 부분
data1 = {'content': '본문1', 'tags':['Python']}

cur.execute('INSERT INTO POST(CONTENT) VALUES(:content)', data1)
con.commit()
PPK = cur.lastrowid 
""" PPK = 1 """

# sns 실제 구동 page 4, ORM
for tag in data1['tags']:
    data = {'tag':tag}

    cur.execute('''
                SELECT COUNT(*) 
                FROM HASHTAG 
                WHERE NAME = :tag -- :뒤에 값이 key값이다.
                ''', data)
    if cur.fetchone()[0] == 0:
        cur.execute('''
             INSERT INTO HASHTAG(NAME) VALUES(:tag)
                    ''', data)
        con.commit()
    cur.execute('''
                SELECT PK 
                FROM HASHTAG 
                WHERE NAME = :tag 
                LIMIT 0,1 -- 여러개가 있을 수 있으니 한 개만 추출
                ''', data)
    TPK = cur.fetchone()[0]
    """ TPK = 1 """

    # 위에 부분 tag를 넣으면 PK 리턴
    cur.execute('INSERT INTO POSTTAG VALUES(?,?)', [PPK, TPK])
    cur.execute('''
                UPDATE HASHTAG SET COUNT = COUNT + 1
                WHERE PK = ?
                ''', [TPK])
    con.commit()

In [98]:
cur.execute('select * from hashtag')
cur.fetchall()

[(1, 1, 'Python')]

In [ ]:
# 숙제: 수정 서비스 로직 구현

In [36]:
# ORM을 위한 라이브러리 설치
! pip install SQLAlchemy

In [38]:
# 엔진 만들기
from sqlalchemy import create_engine, MetaData
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text

In [40]:
# echo를 True로 설정하면 log에 SQL이 기록된다.
# engine을 만드는 순간 실행된다.
engine = create_engine('sqlite:///:memory:', echo = True)

In [41]:
# 사후에 엔진을 바인딩 시킬 수 있다.
meta = MetaData()

In [43]:
# 기존 SQLites와 다르게 테이블을 구성할 테이블 명과 열을 정의하면 간단하게 DB를 만들 수있다.
user = Table('USER', meta, 
             Column('PK', Integer, primary_key = True),
             Column('NAME', Text, nullable = True))

In [44]:
meta.create_all(engine)

2023-09-05 12:24:40,646 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-05 12:24:40,648 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2023-09-05 12:24:40,648 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-05 12:24:40,649 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("USER")
2023-09-05 12:24:40,649 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-05 12:24:40,650 INFO sqlalchemy.engine.Engine 
CREATE TABLE "USER" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT, 
	PRIMARY KEY ("PK")
)


2023-09-05 12:24:40,652 INFO sqlalchemy.engine.Engine [no key 0.00049s] ()
2023-09-05 12:24:40,653 INFO sqlalchemy.engine.Engine COMMIT


In [45]:
meta.tables

FacadeDict({'USER': Table('USER', MetaData(), Column('PK', Integer(), table=<USER>, primary_key=True, nullable=False), Column('NAME', Text(), table=<USER>), schema=None)})

In [46]:
# meta의 자료를 저장하지 않고 clear()를 해버리면 만들어둔 테이블이 날라간다.
meta.clear()

In [47]:
meta.tables

FacadeDict({})

In [48]:
meta.create_all(engine)

2023-09-05 12:26:19,316 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-05 12:26:19,316 INFO sqlalchemy.engine.Engine COMMIT


In [49]:
# reflect를 사용하면 방금 다 날린 테이블을 다시 불러온다
meta.reflect(engine)

2023-09-05 12:27:15,273 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2023-09-05 12:27:15,274 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-05 12:27:15,275 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("USER")
2023-09-05 12:27:15,276 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-05 12:27:15,277 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2023-09-05 12:27:15,278 INFO sqlalchemy.engine.Engine [raw sql] ('USER',)
2023-09-05 12:27:15,279 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("USER")
2023-09-05 12:27:15,280 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-05 12:27:15,281 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("USER")
2023-09-05 12:27:15,282 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-05 12:27:15,282 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite

In [50]:
meta.tables

FacadeDict({'USER': Table('USER', MetaData(), Column('PK', INTEGER(), table=<USER>, primary_key=True, nullable=False), Column('NAME', TEXT(), table=<USER>), schema=None)})

In [51]:
type(user), dir(user)

(sqlalchemy.sql.schema.Table,
 ['__bool__',
  '__class__',
  '__class_getitem__',
  '__delattr__',
  '__dict__',
  '__dir__',
  '__doc__',
  '__eq__',
  '__format__',
  '__ge__',
  '__getattribute__',
  '__getstate__',
  '__gt__',
  '__hash__',
  '__init__',
  '__init_subclass__',
  '__invert__',
  '__le__',
  '__lt__',
  '__module__',
  '__ne__',
  '__new__',
  '__nonzero__',
  '__reduce__',
  '__reduce_ex__',
  '__repr__',
  '__setattr__',
  '__sizeof__',
  '__slots__',
  '__str__',
  '__subclasshook__',
  '__visit_name__',
  '__weakref__',
  '_all_selected_columns',
  '_annotate',
  '_annotations',
  '_annotations_cache_key',
  '_anonymous_fromclause',
  '_assert_no_memoizations',
  '_autoincrement_column',
  '_autoload',
  '_cache_key_traversal',
  '_clone',
  '_cloned_set',
  '_cols_populated',
  '_columns',
  '_compile_w_cache',
  '_compiler',
  '_compiler_dispatch',
  '_constructor',
  '_copy_internals',
  '_deannotate',
  '_dialect_kwargs_traverse_internals',
  '_execute_on_con

In [52]:
# SQLites와 비슷하게 insert 구문을 통해 데이터를 넣는데, 
# 객체를 만들었기 때문에 양식에 맞춰서 넣기만 하면 된다.
user1 = user.insert().values(NAME = '아무개')

In [57]:
# 실행된 결과가 어떻게 컴파일된건지 알기위해서 compile()을 사용한다.
print(user1.compile(), user1.compile().params)

INSERT INTO "USER" ("NAME") VALUES (:NAME) {'NAME': '아무개'}
